# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

C:\Users\cvalentine\AppData\Local\anaconda3\envs\learn-env\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.2) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_test = pd.read_csv('data/y_test.csv')
y_val = pd.read_csv('data/y_val.csv')

In [3]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

As a refresher, preview the training data: 

In [4]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

Here, we're calling .shape on our training data so that we can use the result as n_features, so we know how big to make our input layer.

In [5]:
# How big input layer?
n_features = (X_train.shape[1],)
print(n_features)

(296,)


Create your baseline model. Yo will notice is exihibits strange behavior.

*Note:* When you run this model or other models later on, you may get a comment from tf letting you about optimizing your GPU

In [7]:
# Baseline model
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))

# Hidden layer with 50 units
baseline_model.add(layers.Dense(50, activation='relu'))

# Output layer
baseline_model.add(layers.Dense(1, activation='linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 879us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 940us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [=========================

Epoch 69/150
33/33 [==============================] - 0s 939us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 970us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 970us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 939us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 909us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 909us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [========

Epoch 136/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 137/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [===========

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [8]:
# Numeric column names
numeric_columns = X_train_numeric.columns 

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns=numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.fit_transform(X_val_numeric), columns=numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.fit_transform(X_test_numeric), columns=numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [9]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 
Again, you may get some strange behavior.

In [11]:
# Train the model
normalized_input_model.fit(X_train, y_train, batch_size=32, epochs=150, validation_data=(X_val,y_val))

Epoch 1/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 909us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [===========================

Epoch 69/150
33/33 [==============================] - 0s 970us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 941us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 1000us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 909us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 940us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 879us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 879us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [=====

33/33 [==============================] - 0s 909us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 137/150
33/33 [==============================] - 0s 970us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 940us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 969us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 909us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 878us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 909us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 970us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [=========

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [12]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.fit_transform(y_val)
y_test_scaled = ss_y.fit_transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [13]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(X_train, y_train_scaled, batch_size=32, epochs=150, validation_data=(X_val,y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 2ms/step - loss: 0.5129 - mse: 0.5129 - val_loss: 0.2168 - val_mse: 0.2168
Epoch 2/150
33/33 [==============================] - 0s 1ms/step - loss: 0.2641 - mse: 0.2641 - val_loss: 0.1666 - val_mse: 0.1666
Epoch 3/150
33/33 [==============================] - 0s 1ms/step - loss: 0.2074 - mse: 0.2074 - val_loss: 0.2250 - val_mse: 0.2250
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1807 - mse: 0.1807 - val_loss: 0.1504 - val_mse: 0.1504
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1651 - mse: 0.1651 - val_loss: 0.1410 - val_mse: 0.1410
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1452 - mse: 0.1452 - val_loss: 0.1330 - val_mse: 0.1330
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1324 - mse: 0.1324 - val_loss: 0.1301 - val_mse: 0.1301
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1201 - m

Epoch 63/150
33/33 [==============================] - 0s 970us/step - loss: 0.0264 - mse: 0.0264 - val_loss: 0.1222 - val_mse: 0.1222
Epoch 64/150
33/33 [==============================] - 0s 939us/step - loss: 0.0260 - mse: 0.0260 - val_loss: 0.1114 - val_mse: 0.1114
Epoch 65/150
33/33 [==============================] - 0s 939us/step - loss: 0.0253 - mse: 0.0253 - val_loss: 0.1106 - val_mse: 0.1106
Epoch 66/150
33/33 [==============================] - 0s 939us/step - loss: 0.0251 - mse: 0.0251 - val_loss: 0.1110 - val_mse: 0.1110
Epoch 67/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0250 - mse: 0.0250 - val_loss: 0.1096 - val_mse: 0.1096
Epoch 68/150
33/33 [==============================] - 0s 1000us/step - loss: 0.0250 - mse: 0.0250 - val_loss: 0.1099 - val_mse: 0.1099
Epoch 69/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0239 - mse: 0.0239 - val_loss: 0.1093 - val_mse: 0.1093
Epoch 70/150
33/33 [==============================] - 0s 940us/st

33/33 [==============================] - 0s 1000us/step - loss: 0.0133 - mse: 0.0133 - val_loss: 0.1149 - val_mse: 0.1149
Epoch 125/150
33/33 [==============================] - 0s 909us/step - loss: 0.0131 - mse: 0.0131 - val_loss: 0.1140 - val_mse: 0.1140
Epoch 126/150
33/33 [==============================] - 0s 909us/step - loss: 0.0127 - mse: 0.0127 - val_loss: 0.1131 - val_mse: 0.1131
Epoch 127/150
33/33 [==============================] - 0s 910us/step - loss: 0.0126 - mse: 0.0126 - val_loss: 0.1133 - val_mse: 0.1133
Epoch 128/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0126 - mse: 0.0126 - val_loss: 0.1135 - val_mse: 0.1135
Epoch 129/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0125 - mse: 0.0125 - val_loss: 0.1137 - val_mse: 0.1137
Epoch 130/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0123 - mse: 0.0123 - val_loss: 0.1146 - val_mse: 0.1146
Epoch 131/150
33/33 [==============================] - 0s 980us/step - los

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [14]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 938us/step - loss: 0.0099 - mse: 0.0099


[0.009933228604495525, 0.009933228604495525]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [15]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 667us/step - loss: 0.1187 - mse: 0.1187


[0.11866244673728943, 0.11866244673728943]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [16]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))

33048.21524007698

Great. Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'` 
  
[Documentation on the He Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/HeNormal)

In [17]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, activation='relu', input_shape=(n_features), kernel_initializer='he_normal'))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
he_model.fit(X_train, y_train_scaled, batch_size=32, epochs=150, validation_data=(X_val,y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 2ms/step - loss: 0.4803 - mse: 0.4803 - val_loss: 0.2171 - val_mse: 0.2171
Epoch 2/150
33/33 [==============================] - 0s 1ms/step - loss: 0.2644 - mse: 0.2644 - val_loss: 0.1868 - val_mse: 0.1868
Epoch 3/150
33/33 [==============================] - 0s 969us/step - loss: 0.2007 - mse: 0.2007 - val_loss: 0.1579 - val_mse: 0.1579
Epoch 4/150
33/33 [==============================] - 0s 970us/step - loss: 0.1684 - mse: 0.1684 - val_loss: 0.1523 - val_mse: 0.1523
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1519 - mse: 0.1519 - val_loss: 0.1431 - val_mse: 0.1431
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1348 - mse: 0.1348 - val_loss: 0.1350 - val_mse: 0.1350
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1285 - mse: 0.1285 - val_loss: 0.1357 - val_mse: 0.1357
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1157

Epoch 63/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0232 - mse: 0.0232 - val_loss: 0.1169 - val_mse: 0.1169
Epoch 64/150
33/33 [==============================] - 0s 970us/step - loss: 0.0230 - mse: 0.0230 - val_loss: 0.1178 - val_mse: 0.1178
Epoch 65/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0224 - mse: 0.0224 - val_loss: 0.1172 - val_mse: 0.1172
Epoch 66/150
33/33 [==============================] - 0s 909us/step - loss: 0.0222 - mse: 0.0222 - val_loss: 0.1155 - val_mse: 0.1155
Epoch 67/150
33/33 [==============================] - 0s 940us/step - loss: 0.0220 - mse: 0.0220 - val_loss: 0.1165 - val_mse: 0.1165
Epoch 68/150
33/33 [==============================] - 0s 939us/step - loss: 0.0217 - mse: 0.0217 - val_loss: 0.1176 - val_mse: 0.1176
Epoch 69/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0213 - mse: 0.0213 - val_loss: 0.1154 - val_mse: 0.1154
Epoch 70/150
33/33 [==============================] - 0s 939us/step 

33/33 [==============================] - 0s 868us/step - loss: 0.0113 - mse: 0.0113 - val_loss: 0.1173 - val_mse: 0.1173
Epoch 125/150
33/33 [==============================] - 0s 909us/step - loss: 0.0111 - mse: 0.0111 - val_loss: 0.1157 - val_mse: 0.1157
Epoch 126/150
33/33 [==============================] - 0s 925us/step - loss: 0.0110 - mse: 0.0110 - val_loss: 0.1173 - val_mse: 0.1173
Epoch 127/150
33/33 [==============================] - 0s 932us/step - loss: 0.0110 - mse: 0.0110 - val_loss: 0.1171 - val_mse: 0.1171
Epoch 128/150
33/33 [==============================] - 0s 897us/step - loss: 0.0109 - mse: 0.0109 - val_loss: 0.1158 - val_mse: 0.1158
Epoch 129/150
33/33 [==============================] - 0s 939us/step - loss: 0.0107 - mse: 0.0107 - val_loss: 0.1200 - val_mse: 0.1200
Epoch 130/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0105 - mse: 0.0105 - val_loss: 0.1169 - val_mse: 0.1169
Epoch 131/150
33/33 [==============================] - 0s 848us/step - 

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [18]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 484us/step - loss: 0.0084 - mse: 0.0084


[0.008380772545933723, 0.008380772545933723]

Evaluate the model (`he_model`) on validate data (`X_val` and `y_val_scaled`)

In [19]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 775us/step - loss: 0.1169 - mse: 0.1169


[0.11688125878572464, 0.11688125878572464]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 
  
[Documentation on the Lecun Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/LecunNormal)

In [20]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, kernel_initializer='lecun_normal', activation='relu', input_shape=n_features))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 3ms/step - loss: 0.5048 - mse: 0.5048 - val_loss: 0.2379 - val_mse: 0.2379
Epoch 2/150
33/33 [==============================] - 0s 1ms/step - loss: 0.2604 - mse: 0.2604 - val_loss: 0.1946 - val_mse: 0.1946
Epoch 3/150
33/33 [==============================] - 0s 1ms/step - loss: 0.2028 - mse: 0.2028 - val_loss: 0.1645 - val_mse: 0.1645
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1819 - mse: 0.1819 - val_loss: 0.1934 - val_mse: 0.1934
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1579 - mse: 0.1579 - val_loss: 0.1543 - val_mse: 0.1543
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1375 - mse: 0.1375 - val_loss: 0.1493 - val_mse: 0.1493
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1257 - mse: 0.1257 - val_loss: 0.1471 - val_mse: 0.1471
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1194 - m

Epoch 63/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0223 - mse: 0.0223 - val_loss: 0.1563 - val_mse: 0.1563
Epoch 64/150
33/33 [==============================] - 0s 909us/step - loss: 0.0219 - mse: 0.0219 - val_loss: 0.1552 - val_mse: 0.1552
Epoch 65/150
33/33 [==============================] - 0s 879us/step - loss: 0.0220 - mse: 0.0220 - val_loss: 0.1573 - val_mse: 0.1573
Epoch 66/150
33/33 [==============================] - 0s 909us/step - loss: 0.0218 - mse: 0.0218 - val_loss: 0.1602 - val_mse: 0.1602
Epoch 67/150
33/33 [==============================] - 0s 909us/step - loss: 0.0212 - mse: 0.0212 - val_loss: 0.1570 - val_mse: 0.1570
Epoch 68/150
33/33 [==============================] - 0s 939us/step - loss: 0.0208 - mse: 0.0208 - val_loss: 0.1578 - val_mse: 0.1578
Epoch 69/150
33/33 [==============================] - 0s 1000us/step - loss: 0.0205 - mse: 0.0205 - val_loss: 0.1575 - val_mse: 0.1575
Epoch 70/150
33/33 [==============================] - 0s 909us/

33/33 [==============================] - 0s 879us/step - loss: 0.0113 - mse: 0.0113 - val_loss: 0.1663 - val_mse: 0.1663
Epoch 125/150
33/33 [==============================] - 0s 879us/step - loss: 0.0108 - mse: 0.0108 - val_loss: 0.1676 - val_mse: 0.1676
Epoch 126/150
33/33 [==============================] - 0s 939us/step - loss: 0.0108 - mse: 0.0108 - val_loss: 0.1675 - val_mse: 0.1675
Epoch 127/150
33/33 [==============================] - 0s 879us/step - loss: 0.0108 - mse: 0.0108 - val_loss: 0.1677 - val_mse: 0.1677
Epoch 128/150
33/33 [==============================] - 0s 848us/step - loss: 0.0106 - mse: 0.0106 - val_loss: 0.1676 - val_mse: 0.1676
Epoch 129/150
33/33 [==============================] - 0s 909us/step - loss: 0.0105 - mse: 0.0105 - val_loss: 0.1678 - val_mse: 0.1678
Epoch 130/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0106 - mse: 0.0106 - val_loss: 0.1669 - val_mse: 0.1669
Epoch 131/150
33/33 [==============================] - 0s 879us/step - 

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [21]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 636us/step - loss: 0.0084 - mse: 0.0084


[0.00842274073511362, 0.00842274073511362]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [22]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 778us/step - loss: 0.1706 - mse: 0.1706


[0.17057731747627258, 0.17057731747627258]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

[Documentation on the RMS Prop Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/experimental/RMSprop)

In [23]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
rmsprop_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 2ms/step - loss: 0.4530 - mse: 0.4530 - val_loss: 0.2142 - val_mse: 0.2142
Epoch 2/150
33/33 [==============================] - 0s 1000us/step - loss: 0.2152 - mse: 0.2152 - val_loss: 0.1710 - val_mse: 0.1710
Epoch 3/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1747 - mse: 0.1747 - val_loss: 0.1550 - val_mse: 0.1550
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1472 - mse: 0.1472 - val_loss: 0.1648 - val_mse: 0.1648
Epoch 5/150
33/33 [==============================] - 0s 970us/step - loss: 0.1351 - mse: 0.1351 - val_loss: 0.1419 - val_mse: 0.1419
Epoch 6/150
33/33 [==============================] - 0s 970us/step - loss: 0.1216 - mse: 0.1216 - val_loss: 0.1407 - val_mse: 0.1407
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1131 - mse: 0.1131 - val_loss: 0.1365 - val_mse: 0.1365
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1

Epoch 63/150
33/33 [==============================] - 0s 909us/step - loss: 0.0209 - mse: 0.0209 - val_loss: 0.1347 - val_mse: 0.1347
Epoch 64/150
33/33 [==============================] - 0s 940us/step - loss: 0.0208 - mse: 0.0208 - val_loss: 0.1345 - val_mse: 0.1345
Epoch 65/150
33/33 [==============================] - 0s 940us/step - loss: 0.0202 - mse: 0.0202 - val_loss: 0.1360 - val_mse: 0.1360
Epoch 66/150
33/33 [==============================] - 0s 970us/step - loss: 0.0199 - mse: 0.0199 - val_loss: 0.1341 - val_mse: 0.1341
Epoch 67/150
33/33 [==============================] - 0s 879us/step - loss: 0.0198 - mse: 0.0198 - val_loss: 0.1360 - val_mse: 0.1360
Epoch 68/150
33/33 [==============================] - 0s 939us/step - loss: 0.0191 - mse: 0.0191 - val_loss: 0.1378 - val_mse: 0.1378
Epoch 69/150
33/33 [==============================] - 0s 939us/step - loss: 0.0189 - mse: 0.0189 - val_loss: 0.1363 - val_mse: 0.1363
Epoch 70/150
33/33 [==============================] - 0s 849us

33/33 [==============================] - 0s 939us/step - loss: 0.0095 - mse: 0.0095 - val_loss: 0.1395 - val_mse: 0.1395
Epoch 125/150
33/33 [==============================] - 0s 818us/step - loss: 0.0096 - mse: 0.0096 - val_loss: 0.1403 - val_mse: 0.1403
Epoch 126/150
33/33 [==============================] - 0s 970us/step - loss: 0.0094 - mse: 0.0094 - val_loss: 0.1387 - val_mse: 0.1387
Epoch 127/150
33/33 [==============================] - 0s 1000us/step - loss: 0.0092 - mse: 0.0092 - val_loss: 0.1397 - val_mse: 0.1397
Epoch 128/150
33/33 [==============================] - 0s 908us/step - loss: 0.0091 - mse: 0.0091 - val_loss: 0.1393 - val_mse: 0.1393
Epoch 129/150
33/33 [==============================] - 0s 879us/step - loss: 0.0091 - mse: 0.0091 - val_loss: 0.1423 - val_mse: 0.1423
Epoch 130/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0088 - mse: 0.0088 - val_loss: 0.1423 - val_mse: 0.1423
Epoch 131/150
33/33 [==============================] - 0s 909us/step -

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [24]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 1ms/step - loss: 0.0077 - mse: 0.0077


[0.007662020158022642, 0.007662020158022642]

Evaluate the model (`rmsprop_model`) on training data (`X_val` and `y_val_scaled`) 

In [25]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 636us/step - loss: 0.1445 - mse: 0.1445


[0.14451682567596436, 0.14451682567596436]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric

[Documentation on the Adam Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam)

In [26]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
adam_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse'])

# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 2ms/step - loss: 0.5716 - mse: 0.5716 - val_loss: 0.2536 - val_mse: 0.2536
Epoch 2/150
33/33 [==============================] - 0s 1ms/step - loss: 0.2699 - mse: 0.2699 - val_loss: 0.1850 - val_mse: 0.1850
Epoch 3/150
33/33 [==============================] - 0s 1ms/step - loss: 0.2145 - mse: 0.2145 - val_loss: 0.1660 - val_mse: 0.1660
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1885 - mse: 0.1885 - val_loss: 0.1476 - val_mse: 0.1476
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1659 - mse: 0.1659 - val_loss: 0.1537 - val_mse: 0.1537
Epoch 6/150
33/33 [==============================] - 0s 909us/step - loss: 0.1548 - mse: 0.1548 - val_loss: 0.1365 - val_mse: 0.1365
Epoch 7/150
33/33 [==============================] - 0s 939us/step - loss: 0.1426 - mse: 0.1426 - val_loss: 0.1354 - val_mse: 0.1354
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1372

Epoch 63/150
33/33 [==============================] - 0s 939us/step - loss: 0.0283 - mse: 0.0283 - val_loss: 0.1393 - val_mse: 0.1393
Epoch 64/150
33/33 [==============================] - 0s 879us/step - loss: 0.0280 - mse: 0.0280 - val_loss: 0.1385 - val_mse: 0.1385
Epoch 65/150
33/33 [==============================] - 0s 939us/step - loss: 0.0279 - mse: 0.0279 - val_loss: 0.1383 - val_mse: 0.1383
Epoch 66/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0271 - mse: 0.0271 - val_loss: 0.1375 - val_mse: 0.1375
Epoch 67/150
33/33 [==============================] - 0s 916us/step - loss: 0.0271 - mse: 0.0271 - val_loss: 0.1394 - val_mse: 0.1394
Epoch 68/150
33/33 [==============================] - 0s 939us/step - loss: 0.0260 - mse: 0.0260 - val_loss: 0.1395 - val_mse: 0.1395
Epoch 69/150
33/33 [==============================] - 0s 939us/step - loss: 0.0260 - mse: 0.0260 - val_loss: 0.1380 - val_mse: 0.1380
Epoch 70/150
33/33 [==============================] - 0s 879us/s

33/33 [==============================] - 0s 924us/step - loss: 0.0133 - mse: 0.0133 - val_loss: 0.1498 - val_mse: 0.1498
Epoch 125/150
33/33 [==============================] - 0s 975us/step - loss: 0.0131 - mse: 0.0131 - val_loss: 0.1484 - val_mse: 0.1484
Epoch 126/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0132 - mse: 0.0132 - val_loss: 0.1501 - val_mse: 0.1501
Epoch 127/150
33/33 [==============================] - 0s 944us/step - loss: 0.0129 - mse: 0.0129 - val_loss: 0.1487 - val_mse: 0.1487
Epoch 128/150
33/33 [==============================] - 0s 879us/step - loss: 0.0129 - mse: 0.0129 - val_loss: 0.1496 - val_mse: 0.1496
Epoch 129/150
33/33 [==============================] - 0s 938us/step - loss: 0.0128 - mse: 0.0128 - val_loss: 0.1497 - val_mse: 0.1497
Epoch 130/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0126 - mse: 0.0126 - val_loss: 0.1509 - val_mse: 0.1509
Epoch 131/150
33/33 [==============================] - 0s 939us/step - lo

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [27]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 515us/step - loss: 0.0100 - mse: 0.0100


[0.009959469549357891, 0.009959469549357891]

Evaluate the model (`adam_model`) on training data (`X_val` and `y_val_scaled`) 

In [28]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 668us/step - loss: 0.1520 - mse: 0.1520


[0.15198072791099548, 0.15198072791099548]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [30]:
# Evaluate the best model on test data
rmsprop_model.evaluate(X_test, y_test_scaled)

5/5 [==============================] - 0s 801us/step - loss: 0.1335 - mse: 0.1335


[0.13351638615131378, 0.13351638615131378]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [31]:
# Generate predictions on test data
y_test_pred_scaled = rmsprop_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
np.sqrt(mean_squared_error(y_test, y_test_pred))

34926.93982733494

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.